### Read the different datasets

In [1]:
import pandas as pd
import plotly.express as px
import yaml
import re
import numpy as np
import panel as pn
import statsmodels
from bokeh.io import output_notebook
from pathlib import Path
from scipy.stats import chi2_contingency
from IPython.display import Markdown as md

output_notebook()
pn.extension('plotly')

Loading BokehJS ...

### Research question: Does cancer occur more often among families living in poverty than families living above minimum wage?

#### Hypothesis 1
#### Cancer is more prevelant among families living in poverty


#### Hypothesis 2
#### Cancer occurs more often in provinces with many families receiving social assistance

In [2]:
def get_config():
    with open("config.yaml","r") as stream:
        config = yaml.safe_load(stream)
    return config

config = get_config()
data_dir = config["datadir"]

#The data contains some missing values, a row named Periode. This refers to the year. However,
# since the years are used as an index this rows will only contain NaN values and is thus removed.
cancer_cases = pd.read_excel(Path(data_dir + "NKR-export-data-14_01_2022_13_38_53.xlsx"), 
                             header=0, skiprows=8, index_col=0)

#The following code removes the first NaN column in windows. Somehow, drop.na() works on the linux systems, but not
#on my home computer.
cancer_cases = cancer_cases.reset_index()
cancer_cases = cancer_cases.set_index("Geografie")
cancer_cases = cancer_cases.iloc[1:,:]
cancer_cases = cancer_cases.reset_index()
cancer_cases = cancer_cases.rename(columns={"Geografie": 'Year'})
cancer_cases = cancer_cases.set_index("Year")

#Only specific columns are used. All  other columns are excluded while 
poverties = pd.read_excel(Path(data_dir + "regionaal_gemeente_pc4.ods"), 
                          engine="odf", sheet_name=1, usecols=["jaar", "gm_naam", "p_totaal", "p_uitkering"])


#Two datasets of municipalities are needed so that for all but one municipality can be searched in which province
#the municipality is located.
municipalities2010 = pd.read_excel(Path(data_dir + "2010-gemeenten-per-provincie.xls"), header=0)

municipalities2018 = pd.read_excel(Path(data_dir + "2018-gemeenten-per-provincie.xls"), header=0)

/home/rvandepol/.local/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
#The first thing that needs to be done is to add the province to a column in poverties. 
#The poverty data is given per province, and not per municipality.
#Therefore, instead of municipality, the poverty data needs to be viewed on a municipality level.
#This also means that the percentages of poverty per municipality have to be averaged out, to obtain the information
#per province. The average poverty percentage of each province will be calculated. 
def find_municipality(x):
    if x in municipalities2010.Gemcodel.values:
        return "".join(municipalities2010[municipalities2010.Gemcodel == x].provcodel.values)
    
    elif x in municipalities2018.Gemeentenaam.values:
        return "".join(municipalities2018[municipalities2018.Gemeentenaam == x].Provincienaam.values)
    
    #Menameradiel is the only municipality not in either of both dataframes. It is not exactly clean, but
    #returning the province in this way works perfectly.
    elif x == "Menameradiel":
        return "Friesland"

#Searches for the province the municipalties are located using the find_municipality function.
poverties["Provincie"] = list(map(find_municipality, poverties.gm_naam))

In [4]:
provinces_cancer = poverties.groupby(by=[poverties.jaar, 
                         poverties.Provincie]).agg({'p_totaal':'mean', 'p_uitkering':'mean'}).reset_index()

In [5]:
#Returns the cancer cases as strings, instead of integers or values still containing brackets.
def to_str(var): 
    return str(list(np.reshape(np.asarray(var), (1, np.size(var)))[0]))[1:-1]


#Function used in conjuction with map to add the cancer cases to the correct province and year
def add_cancer_cases(x, y):
    return to_str(cancer_cases[(cancer_cases.index == str(x))][str(y)].values)


provinces_cancer["Cancer_Incidents"] = list(map(add_cancer_cases, provinces_cancer.jaar, provinces_cancer.Provincie))
provinces_cancer.Cancer_Incidents = pd.to_numeric(provinces_cancer.Cancer_Incidents).astype("int64")
#Since the Poverty and Social assitance values are given per municipality instead of Province, the mean of
#all municipalites in a province is calculated for both values.
provinces_cancer = provinces_cancer.rename(columns={"jaar":"Year", "Provincie":"Province",
                                "p_totaal":"P(Poverty)", "p_uitkering":"P(Social Assistance)"})

In [22]:
fig = px.scatter(provinces_cancer, x="P(Poverty)", y="P(Social Assistance)", 
                 animation_frame="Year", animation_group="Province",
                 size="Cancer_Incidents", color="Province", hover_name="Province",
                 range_x=[0.04, 0.08], range_y=[0.1,0.3], trendline="ols",
                 title = f"Bubble Chart of the P value of social assitance against the"+
                         "p value of poverty of Dutch municipalities <br>of the year",
                 labels = {'P(Poverty)':"P(Poverty)<br><sup>Fruit sales in the month of January</sup>"})

fig = px.scatter(provinces_cancer, x="P(Poverty)", y="Cancer_Incidents")
fig["layout"].pop("updatemenus")
fig.show()

In [7]:
def bubble_plot(year):
    hover_data = hover_data={"Province":False,
                             "Year":False,
                             "P(Poverty)":False,
                             "P(Social Assistance)":False,
                             "Cancer_Incidents":True}
    
    fig = px.scatter(provinces_cancer.query(f"Year=={year}"), x="P(Poverty)", y="P(Social Assistance)", 
                     width=800, height=600, size="Cancer_Incidents", color="Province", hover_name="Province", 
                     animation_frame="Year", hover_data=hover_data,
                     title = "Bubble Chart of the P value of social assitance against the p value of" + "<br>" +
                         f"poverty of Dutch municipalities of the year {year}<br>",
                     labels = {'P(Poverty)':"P(Poverty)<br><sup>This bubble plot shows the relation between P(Poverty)and P(Social Assistance) " +
                           "<br>" + "with P being the probability a random person falls in that category." 
                           "<br>The size of the bubbles corresponds to the number of cancer cases in that province</sup>"})
    return fig

start = provinces_cancer.Year.unique().tolist()[0]
end = provinces_cancer.Year.unique().tolist()[-1]

slider = pn.widgets.IntSlider(name='Slider', start=start, end=end, step=2, value=2011)


pn.interact(bubble_plot, year=slider)

Column
    [0] Column
        [0] IntSlider(end=2017, name='Slider', start=2011, step=2, value=2011)
    [1] Row
        [0] Plotly(Figure, name='interactive00102')

The bubble plot shows that, unsuprisingly, a strong correlation between poverty probablity and social assistance probability exists. What is supsrising though is that Flevoland has both the highest poverty probabilty
and social assistance probabilty. However, Flevoland has only very few cancer cases. Something simillar is seen for the other provinces. The provinces with the highest cancer cases actually have (nearly) the lowest poverty and social assistance probabilities, as all big bubbles are on the left side of the plot. 
An explanation for this may actually be that, because those people live in poverty and receive social assistance, they are less likely to visit the hospital for check-ups. Meaning that, for those people, cancer is not detected and therefore would not be included in the data. It might be interesting for future research to investigate at what stage the cancer is discovered. It might very well be that cancer is discovered at a higher stage when the poverty probability is high and that the cancer stage is low when the poverty probability is low.

In [8]:
def boxplot(var):
    fig = px.box(provinces_cancer, x="Year", y=var, 
                 title = f"Boxplots of {var} for the years 2011 untill 2017 <br>with a 2 year interval<br>",
                 labels = {'Year':f"Year<br><sup>Boxplots explaining the variance of {var} <br>"
                           "of all provinces in the Netherlands</sup>"})
    return fig


var = pn.widgets.Select(name="Variable", 
                            options=provinces_cancer.columns.tolist()[2::], 
                            value=provinces_cancer.columns.tolist()[2])

pn.interact(boxplot, var=var)

Column
    [0] Column
        [0] Select(name='Variable', options=['P(Poverty)', ...], value='P(Poverty)')
    [1] Row
        [0] Plotly(Figure, name='interactive00112')

The plot above shows multiple boxplots for each year, for either cancer incidents, the poverty probability and the social assistance probability in that year in the entire country of the Netherlands, regardless of province.
The boxplots of the poverty probabilty differs over the course of the year. The poverty probabilty is the highest in the year 2013, and the lowest in 2017. The variance also differs, it being the highest in 2013 and the smallest in 2015.

The boxplots for the social assistance probability also differ for the different years. It is the lowest in 2017 and the highest in 2013. The variance is the highest in 2013, just like the boxplot of the poverty probability. However, the smallest probability for social assistance is in 2017 instead of 2015.

Finally, the boxplots for the cancer incidents seems to remain approximately the same. A slight increase can be spotted as the years increase, but this does not seem to be significant. Further statistical tests are required.

### Statistical Tests

In [18]:
pearson_correlations = pd.DataFrame(provinces_cancer.groupby("Year")[["P(Poverty)", "Cancer_Incidents"]].corr("pearson"))

spearman_correlations = pd.DataFrame(provinces_cancer.groupby("Year")[["P(Poverty)", "Cancer_Incidents"]].corr("spearman"))

poverty_correlations = pd.DataFrame(provinces_cancer.groupby("Year")[["P(Poverty)", "P(Social Assistance)"]].corr("spearman"))


print(pearson_correlations)
print(spearman_correlations)
print(poverty_correlations)

low_pov, high_pov = provinces_cancer["P(Poverty)"].quantile([0.25, 0.75])
low_cancer, high_cancer = provinces_cancer["Cancer_Incidents"].quantile([0.25, 0.75])

                       P(Poverty)  Cancer_Incidents
Year                                               
2011 P(Poverty)          1.000000         -0.567732
     Cancer_Incidents   -0.567732          1.000000
2013 P(Poverty)          1.000000         -0.575751
     Cancer_Incidents   -0.575751          1.000000
2015 P(Poverty)          1.000000         -0.565929
     Cancer_Incidents   -0.565929          1.000000
2017 P(Poverty)          1.000000         -0.522391
     Cancer_Incidents   -0.522391          1.000000
                       P(Poverty)  Cancer_Incidents
Year                                               
2011 P(Poverty)          1.000000         -0.650350
     Cancer_Incidents   -0.650350          1.000000
2013 P(Poverty)          1.000000         -0.622378
     Cancer_Incidents   -0.622378          1.000000
2015 P(Poverty)          1.000000         -0.608392
     Cancer_Incidents   -0.608392          1.000000
2017 P(Poverty)          1.000000         -0.629371
     Cancer_

The correlation table shows that, contrary to the hypothesis, poverty and cancer seem to be negatively correlated.
For each year, the corrleation between P(Poverty) and Cancer_Incidents are approximately -.56. This means that, to some extent, cancer occurs less among people living in poverety. This may not be accurate however due to the reasons stated above. Unsuprisingly, the correlation plot of poverty and social assistance show strong correlation for each year. With the lowest correlation in 2017 at 0.61 and the highest begin 2011 and 2013 at 0.86. Both the spearman and pearson correlations are calculated. The pearson is used for contiuous variables whereas the spearman is used for values that describe some order. Spearman maybe more valuable in this case because the data is non-linear. The speraman correlation return values of approximately -0.62 for all years. This means that, as the poverty increases the cancer cases actually decrease.

In [10]:
#Determine the quantiles of the povery and cancer columns. These are then used to change the data
#into categorial data. This way the chi2 test can be performed.
#Set cancer and poverty levels based on the quantiles
low_pov, high_pov = provinces_cancer["P(Poverty)"].quantile([0.25, 0.75])
low_cancer, high_cancer = provinces_cancer["Cancer_Incidents"].quantile([0.25, 0.75])

provinces_cancer["pov_level"] = np.where(provinces_cancer["P(Poverty)"] < low_pov, "low", 
                                        np.where(provinces_cancer["P(Poverty)"] > high_pov, "high", "medium"))

provinces_cancer["cancer_level"] = np.where(provinces_cancer["Cancer_Incidents"] < low_cancer, "low", 
                                        np.where(provinces_cancer["Cancer_Incidents"] > high_cancer, 
                                                 "high", "medium"))

contigency = pd.crosstab(provinces_cancer["pov_level"], provinces_cancer["cancer_level"]) 

fig = px.imshow(contigency, text_auto=True, color_continuous_scale="YlGnBu"
                , title="Heatmap of the categories of poverty and cancer incidents")

In [11]:
fig.show()

In [12]:
# Chi-square test of independence. 
c, p, dof, expected = chi2_contingency(contigency) 
# Print the p-value
print("Chi2 p-value:",round(p, 5))

Chi2 p-value: 0.01674


In [16]:
md(f"""With a confidence level of 95% the null hypothesis can be rejected. 
The null hypothesis being that cancer level and poverty level are independent. 
This means that the cancer level and poverty level are dependent on each other.
The exact relationship however is not clear from this. The heatmap shows the combinations of 
poverty and cancer levels. 
From this you can see that provinces with a medium cancer level (between {round(low_cancer)} 
and {round(high_cancer)} cases)
and a medium poverty level (probability between {round(low_pov, 5)} and {round(high_pov, 5)}) occurs 13 times
However, the hypothesis would expect that provinces with a high poverty level 
(probability > {round(high_pov, 4)}) also often have a high cancer level (>{round(high_cancer)}).
The heatmap clearly shows that this, however, is not the case. This is enforced by the correlation table,
which shows that the opposite seems more likely. The negative correlation suggests that high poverty leads to
a reduced cancer level.
""")

With a confidence level of 95% the null hypothesis can be rejected. 
The null hypothesis being that cancer level and poverty level are independent. 
This means that the cancer level and poverty level are dependent on each other.
The exact relationship however is not clear from this. The heatmap shows the combinations of 
poverty and cancer levels. 
From this you can see that provinces with a medium cancer level (between 3770 
and 14313 cases)
and a medium poverty level (probability between 0.04719 and 0.05803) occurs 13 times
However, the hypothesis would expect that provinces with a high poverty level 
(probability > 0.058) also often have a high cancer level (>14313).
The heatmap clearly shows that this, however, is not the case. This is enforced by the correlation table,
which shows that the opposite seems more likely. The negative correlation suggests that high poverty leads to
a reduced cancer level.
